# Reference Tune-up

This notebook shall evolve into a reference tune-up notebook. Currently, it is to showcase the new experiment library and the related helper files and functions.

In [1]:
from laboneq.simple import *

import laboneq_library.experiments.experiment_library as experiments
import laboneq_library.analysis.result_plotting as plotting
from laboneq_library.calibration_helpers import (
    load_qubit_parameters,
    update_qubit_parameters_and_calibration,
)

In [ ]:
emulate = True

In [ ]:
from textwrap import dedent

descriptor = dedent(
    """\
instruments:
  SHFQC:
    - address: DEV12108
      uid: device_shfqc
connections:

  device_shfqc:
    - iq_signal: q0/measure_line
      ports: QACHANNELS/0/OUTPUT
    - acquire_signal: q0/acquire_line
      ports: QACHANNELS/0/INPUT

    - iq_signal: q0/drive_line
      ports: SGCHANNELS/0/OUTPUT

    - iq_signal: q1/measure_line
      ports: QACHANNELS/0/OUTPUT
    - acquire_signal: q1/acquire_line
      ports: QACHANNELS/0/INPUT

    - iq_signal: q1/drive_line
      ports: SGCHANNELS/1/OUTPUT
"""
)
demo_setup = DeviceSetup.from_descriptor(
    descriptor, server_host="10.42.11.10", server_port=8004, setup_name="MySetup"
)

## Apply Calibration from File

In [ ]:
qubit_parameters = load_qubit_parameters()

transmon_list = update_qubit_parameters_and_calibration(
    qubit_parameters, device_setup=demo_setup, database=None
)

### Create and Connect to a QCCS Session 

Establishes the connection to the instruments and readies them for experiments


In [ ]:
# create and connect to a session
session = Session(device_setup=demo_setup)
session.connect(do_emulation=emulate, reset_devices=True)

# Tune-up

## Amplitude Rabi Experiment

In [ ]:
amplitude_sweep = LinearSweepParameter(start=0, stop=1, count=5)
num_averages = 2**10
cal_trace = False

amplitude_rabi = experiments.amplitude_rabi_single(
    qubit=transmon_list[0],
    amplitude_sweep=amplitude_sweep,
    num_averages=num_averages,
    cal_trace=cal_trace,
)

In [ ]:
res = session.run(amplitude_rabi)
show_pulse_sheet("Rabi", res.compiled_experiment)

## Resonator spectroscopy over a full range

In [ ]:
res_spec_full_range = experiments.resonator_spectroscopy_full_range(
    transmon_list[0],
    [4.5e9, 5e9, 5.5e9, 6e9, 6.5e9],
    inner_start=-400e6,
    inner_stop=400e6,
    inner_count=501,
    integration_time=1e-3,
)

In [ ]:
session.connect(do_emulation=emulate, reset_devices=True)
spectroscopy_results = session.run(res_spec_full_range)

In [ ]:
plotting.plot_resonator_spectroscopy_full_range(spectroscopy_results)

## Detuned Ramsey experiment on two qubits

In [ ]:
ramsey_exp_parallel = experiments.ramsey_parallel(
    transmon_list,
    LinearSweepParameter(start=0, stop=40e-6, count=3),
    num_averages=2,
    detuning=50e3,
    cal_trace=True,
)

In [ ]:
session.connect(do_emulation=emulate, reset_devices=True)
results = session.run(ramsey_exp_parallel)

In [ ]:
show_pulse_sheet("ramsey", results.compiled_experiment)